<img src="https://www.hl7.org/fhir/assets/images/fhir-logo-www.png" style="float: left; width: 25%; margin-bottom: 0.5em;">

- [Section 1: The FHIR HTTP API](#Section-1.-The-FHIR-REST-API)
- [Section 2: FHIR Search](#Section-2:-FHIR-Search)
- [Section 3: Search paramater types](#Section-3:-Search-parameter-types)
- [Section 4: Chaining and includes](#Section-4:-Chaining-and-includes)
- [Section 5: Putting it together](#Section-5:-Putting-it-together)
- [Section 6: Bulk export](#Section-6:-Bulk-export)

In [1]:
# save the base url of the FHIR server
base = 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open'

# setup imports
import os
from requests import get
from requests import post
from requests import put
from requests import delete
from requests import head
from IPython.display import IFrame

# a function to print the top x rows and add a newline
def peek(string, line_count=25):
    print(os.linesep.join(string.split(os.linesep)[:line_count]) + '\n')

In [2]:
#(Optional)
# install jsonpointer
!pip install jsonpointer
from jsonpointer import resolve_pointer as resolve

## Section 1. The FHIR REST API

In [3]:
# HL7 FHIR defines a set of "resources" for exchanging information.
IFrame('https://www.hl7.org/fhir/resourcelist.html#tabs', width=1200, height=330)

In [4]:
# Each resource type supports the same set of interactions, categorized in the spec into "instance-level" and "type-level" interactions.
IFrame('https://www.hl7.org/fhir/http.html#operations', width=1200, height=330)

In [5]:
# This notebook focuses on the FHIR Search API, but first we use the "capabilities" interaction to learn about our target server.

# retrieve the server "CapabilityStatement" and print the important bits
response = get(base + '/metadata')
print('Response code: ' + str(response.status_code))
result = response.json()
print('Server: ' + result['name'] + ' ' + result['version'])
print('Security: ' + str(result['rest'][0]['security']))
resources = result['rest'][0]['resource']

supported_types = {r['type']: [i['code'] for i in r['interaction']] for r in resources}

print('Supported types: ')
for k,v in supported_types.items():
    print('  ' + k + ': ' + str(v))

Response code: 200
Server: IBM FHIR Server 4.5.0
Security: {'cors': True}
Supported types: 
  Account: ['read', 'vread', 'history-instance', 'search-type']
  DocumentManifest: ['read', 'vread', 'history-instance', 'search-type']
  CapabilityStatement: ['read', 'vread', 'history-instance', 'search-type']
  Specimen: ['read', 'vread', 'history-instance', 'search-type']
  Location: ['read', 'vread', 'history-instance', 'search-type']
  MedicinalProductPackaged: ['read', 'vread', 'history-instance', 'search-type']
  ExampleScenario: ['read', 'vread', 'history-instance', 'search-type']
  MedicinalProductContraindication: ['read', 'vread', 'history-instance', 'search-type']
  ImagingStudy: ['read', 'vread', 'history-instance', 'search-type']
  EvidenceVariable: ['read', 'vread', 'history-instance', 'search-type']
  CoverageEligibilityResponse: ['read', 'vread', 'history-instance', 'search-type']
  ImmunizationEvaluation: ['read', 'vread', 'history-instance', 'search-type']
  ResearchElementD

## Section 2: FHIR Search

In [6]:
# Now that we know our server supports the "search-type" interaction on all resource types, lets start working with the Patient endpoint.

# query for all Patient resources, then print the HTTP status code and the first 25 lines of the response
response = get(base + '/Patient')
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

# technically you've now performed your first FHIR "search" (just with no parameters)

Response code: 200
Response body: 
{
    "resourceType": "Bundle",
    "id": "3dae0c87-6cbc-465a-8eea-e6768dd09280",
    "type": "searchset",
    "total": 32459,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {
                "resourceType": "Patient",
                "id": "17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
                "meta": {
                    "versionId": "1

In [7]:
# note that results are paged and the "link" field in the response Bundle contains links to the previous, current, and next page of results
for link in response.json().get('link'):
    if link.get('relation') == 'next':
        page2 = get(link.get('url'))        
peek('Second page: \n' + page2.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Second page: 
{
    "resourceType": "Bundle",
    "id": "e4907718-ea58-45e6-b826-6dedc0123392",
    "type": "searchset",
    "total": 32459,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=2"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=3"
        },
        {
            "relation": "previous",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_page=1"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bf809c-1a099ca4-cdb9-4837-8692-51338082cd97",
            "resour

In [8]:
# we can control the number of resources on each page by passing the _count parameter
response = get(base + '/Patient?_count=1')
peek('Single resource per page: \n' + response.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Single resource per page: 
{
    "resourceType": "Bundle",
    "id": "e4d3d60c-ee08-4485-9fd7-99d4badc5e5b",
    "type": "searchset",
    "total": 32459,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {
                "resourceType": "Patient",
                "id": "17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
                "meta": {
                    "versionId": "1",
       

In [9]:
# if you're only interested in the count, you can specify that via either
# A. _count=0 (0 results per page); or
# B. _summary=count

print(get(base + '/Patient' + '?' + '_summary=count').text)

{
    "resourceType": "Bundle",
    "id": "c7b10883-9e40-494d-83ba-49b11619f525",
    "meta": {
        "tag": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
                "code": "SUBSETTED",
                "display": "subsetted"
            }
        ]
    },
    "type": "searchset",
    "total": 32459,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&_summary=count&_page=1"
        }
    ]
}


In [10]:
# if you want a lot of results per page, you can reduce the amount of data returned via the _summary or _elements parameters

# first, lets review the structure of the Patient resource
IFrame('https://hl7.org/fhir/patient.html#resource', width=1200, height=330)

In [11]:
# print the list of top-level elements in the first Patient resource returned
response = get(base + '/Patient')
peek('Normal: \n' + str(response.json().get('entry')[0].get('resource').keys()))

# look for the Σ flag in the Resource Content section of the resource page in the specification for what elements are considered "summary" elements 
response = get(base + '/Patient?' + '_summary=true')
peek('Summary: \n' + str(response.json().get('entry')[0].get('resource').keys()))

# need more control?
# you can use the _elements parameter to ask for specific fields back (although the server should include required fields and modifier fields as well)
response = get(base + '/Patient?' + '_elements=id,gender')
peek('Elements: \n' + str(response.json().get('entry')[0].get('resource').keys()))

Normal: 
dict_keys(['resourceType', 'id', 'meta', 'text', 'extension', 'identifier', 'name', 'telecom', 'gender', 'birthDate', 'address', 'maritalStatus', 'multipleBirthBoolean', 'communication'])

Summary: 
dict_keys(['resourceType', 'id', 'meta', 'identifier', 'name', 'telecom', 'gender', 'birthDate', 'address'])

Elements: 
dict_keys(['resourceType', 'id', 'meta', 'gender'])



In [12]:
# this can add up!

response = get(base + '/Patient?_count=100')
print('Normal: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_summary=true')
print('Summary: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_elements=id,gender,birthDate')
print('Elements: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')


Normal: 	892360 bytes 	(0.396416 s)
Summary: 	516647 bytes 	(0.331421 s)
Elements: 	104260 bytes 	(0.322297 s)


In [13]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
IFrame('https://www.hl7.org/fhir/patient.html#search', width=1200, height=500)

In [14]:
# for example, lets use the search parameter named "gender"
response = get(base + '/Patient' + '?' + 'gender=male')
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text)

Response code: 200
Response body: 
{
    "resourceType": "Bundle",
    "id": "eb159d1b-bf51-40ff-b992-4722682ddc80",
    "type": "searchset",
    "total": 15323,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&gender=male&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=10&gender=male&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
            "resource": {
                "resourceType": "Patient",
                "id": "17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c",
                "meta": {
           

In [15]:
# pro tip: combine your search query with _summary=count to explore the data
print('male:   \t' + str(get(base + '/Patient' + '?' + 'gender=male' + '&' + '_summary=count').json().get('total')))
print('female: \t' + str(get(base + '/Patient' + '?' + 'gender=female' + '&' + '_summary=count').json().get('total')))

# use the "missing" modifier to look for resources that do NOT have a value for the target parameter
response = get(base + '/Patient' + '?' + 'gender:missing=true' + '&' + '_summary=count')
print('missing gender: ' + str(response.json().get('total')))

male:   	15323
female: 	17136
missing gender: 0


## Section 3: Search parameter types

In [16]:
# search parameters have types

# gender is considered a "token" search parameter

# Token search
# this parameter type is common for 'coded' values (Code, Coding, and CodeableConcept) and identifiers
# token values consist of a system and a code, although sometimes the system is implicit (like in the case of gender)
# users can search on the system and code (system|code), the code alone (code), system-less codes (|code), or even the system alone (system|)
response = get(base + '/Patient' + '?' + 'gender=http://hl7.org/fhir/administrative-gender|male' + '&_count=1&_elements=gender')
print('male:\n' + response.text)


# there are also Number, Date/DateTime, String, Reference, Quantity, URI, and Composite parameter types

male:
{
    "resourceType": "Bundle",
    "id": "810abe80-fa63-42df-9711-d511be1ac3ba",
    "meta": {
        "tag": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
                "code": "SUBSETTED",
                "display": "subsetted"
            }
        ]
    },
    "type": "searchset",
    "total": 15323,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&gender=http://hl7.org/fhir/administrative-gender%7Cmale&_elements=gender&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&gender=http://hl7.org/fhir/administrative-gender%7Cmale&_elements=gender&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://clu

In [17]:
# String search
response = get(base + '/Patient' + '?' + 'family=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
        

Smiths:
17598d8e9df-706d630f-784d-473d-a0d2-37af26e3c36e: Smitham
17598dc69c4-fe632367-ce4f-413b-9639-5f409631fdbd: Smith
17598e7faf2-37231ae2-90fa-4668-90cc-9d413e44ff73: Smitham
17598fe2d81-8588df6c-a4e5-486f-8de7-f2a457e1ecbb: Smitham
175991c2ff1-8b5aef81-6ed2-4173-8ee0-3a38612ece43: Smitham
175991de90e-b9364b28-a200-4d19-9bba-1cd4d5d9f037: Heller, Smith
175992f962b-159acf20-e5d5-4370-bf38-8e02eef6d761: Smith
17599326aa9-40e01d7e-1d5b-4f1b-9bed-9c0a0b031df1: Smith
175993420d0-eb6a0968-589a-4a71-a010-19e84056c44e: Smitham
17599375dc6-21078467-b048-499a-b8c4-9f5ddfc6ffd3: Haley, Smitham


In [18]:
# wait, "Smitham" !?

# string search performs a case-insensitive "begins-with" search by default!
# use the modifier ":exact" if you want exact matches (and improved performance)
response = get(base + '/Patient' + '?' + 'family:exact=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
print()

# string search also has a ":contains" modifier
response = get(base + '/Patient' + '?' + 'family:contains=ski' + '&_elements=name')
print('Skis:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))


Smiths:
17598dc69c4-fe632367-ce4f-413b-9639-5f409631fdbd: Smith
175991de90e-b9364b28-a200-4d19-9bba-1cd4d5d9f037: Heller, Smith
175992f962b-159acf20-e5d5-4370-bf38-8e02eef6d761: Smith
17599326aa9-40e01d7e-1d5b-4f1b-9bed-9c0a0b031df1: Smith
175993c2874-5f898109-a18d-4d6e-8cfb-ecd69eb37862: Smith
17599522520-bdb4311f-3765-45f2-bd5d-1f1aca14a931: Smith
17599539a99-bccf02f3-57ef-4399-a4c6-0296aa8356c1: Purdy, Smith
1759954cc00-9def2bdc-a444-4e84-bf77-a0e79e449d19: Rosenbaum, Smith
175995a5564-869be93e-c7c2-4cab-9951-d15063f39975: Wintheiser, Smith
175995ad366-3722d3d9-c56e-4d06-ba32-1b662e18bfbe: Wehner, Smith

Skis:
17598bf6496-92b7682f-d882-4b3c-8e37-178b11cde674: Gusikowski, West
17598c069d0-3b1b68c5-13e0-4c46-b929-1fea6d13454f: Osinski
17598c0689e-e60c055e-c166-406d-a9b0-f6d113292dcc: Gutkowski
17598c10cbc-642eb489-8d83-4220-8591-a31adc7a9189: Gusikowski
17598c3ebc7-63c73ac0-7cc7-492f-b1f5-661c9da0b95e: Gulgowski
17598c45f73-e1ac3687-ae53-4591-b9f7-65deb78c8a1b: Jaskolski
17598c61295-b

In [19]:
# Date search
response = get(base + '/Patient' + '?' + 'birthdate=1984' + '&_elements=birthDate')
print('Born in 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

Born in 1984:
17598c01014-694ba51f-a02d-4686-b956-7d6eb9d7e5fa: 1984-12-05
17598c330b0-817d8bac-b275-449d-8163-0ae095d6a7d5: 1984-03-23
17598d78d31-639632e7-4f79-4b1a-81e8-f462de2c5912: 1984-09-18
17598dc812f-37fd8fe9-e900-4f11-b166-5515fd7b4125: 1984-12-26
17598dda658-9243965e-f6e2-4909-b2f6-89dcd7dcf799: 1984-05-04
17598de13a7-c2c48b25-a1cc-498a-9b98-08629619ae15: 1984-03-04
17598e32314-d67c7847-5ca3-4b0c-86df-195e40626a65: 1984-10-05
17598e54dfe-683137bf-6a96-4b94-9218-7be5ff4ee2fb: 1984-04-21
17598eed38d-eb24c019-4ab9-4f14-9c66-db95a1ed22dc: 1984-10-23
17598f52f36-17f4bfd8-c305-468f-9c38-792a461e28a3: 1984-04-26


In [20]:
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"
response = get(base + '/Patient' + '?' + 'birthdate=eb1984' + '&_elements=birthDate')
print('Born before 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

response = get(base + '/Patient' + '?' + 'birthdate=sa1984' + '&_elements=birthDate')
print('\n' + 'Born after 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

# some servers support ap(approximately equal) as well, although the spec lets the server decide exactly what that means...
response = get(base + '/Patient' + '?' + 'birthdate=ap1984' + '&_elements=birthDate')
print('\n' + 'Born "around" 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

Born before 1984:
17598bf0165-6eda45e2-f9c1-4c4c-9ef2-1776cfbb0a7d: 1976-10-11
17598bf1688-037fcb72-18af-40ce-a96b-c393a595dd49: 1975-07-10
17598bf28f3-257b9fe9-9084-41d6-bb0f-43b4729a7f0d: 1982-01-13
17598bf36c7-fedcedc3-b78c-4688-82ef-622e0cc71b22: 1973-03-03
17598bf6cd6-d8388338-897d-48a0-b334-a877459941b2: 1979-03-01
17598bf3d06-15fa6deb-59ad-4bd2-b8a1-8a2fa606d084: 1924-10-07
17598bf6496-92b7682f-d882-4b3c-8e37-178b11cde674: 1948-12-25
17598bf98b2-789821a7-b30c-48d4-b388-5343a933d8ad: 1972-06-15
17598bfb281-9ca10cb0-6ba0-4691-85f4-424423b1869d: 1970-12-02
17598bfa2da-9c19d3ed-5bef-49ab-b6b0-1d731446ddf6: 1960-11-21

Born after 1984:
17598bed122-d90e0186-8458-4482-a580-5b18b554ed3c: 1990-02-05
17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f: 1986-07-15
17598bf4a5d-185e291b-d0e3-42d3-9b70-eeae60debeec: 2005-11-09
17598bf2ea9-7238cb29-0108-47a7-bf50-c94c6468ad2b: 2020-01-19
17598bf555d-8e5857a0-4da9-4e6c-b9be-8624f80cf080: 1990-01-07
17598bf809c-1a099ca4-cdb9-4837-8692-51338082cd97:

In [21]:
# Reference search
response = get(base + '/Patient?general-practitioner:missing=false&_elements=generalPractitioner,link,managingOrganization&_count=1')
peek('Patients with a general-practitioner:   \n' + response.text, 15)

Patients with a general-practitioner:   
{
    "resourceType": "Bundle",
    "id": "0a9f0316-b9b2-46df-8eed-385560273eb2",
    "meta": {
        "tag": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
                "code": "SUBSETTED",
                "display": "subsetted"
            }
        ]
    },
    "type": "searchset",
    "total": 0,



In [22]:
# since our model doesn't have any reference fields on the Patient resources, lets look at Conditions instead
IFrame('https://hl7.org/fhir/condition.html#resource', width=1200, height=480)

In [23]:
# get all conditions that reference a specific patient
response = get(base + '/Condition' + '?' + 'subject=Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('Conditions for patient 17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('code'))

# when the type of the reference is fixed to a single value, it can be omitted (Patient/x -> x)
response2 = get(base + '/Condition' + '?' + 'patient=17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response2.json().get('entry')))

# a reference to a resource's full url on the server should be equivalent to the relative reference format mentioned above
response3 = get(base + '/Condition' + '?' + 'patient=' + base + '/Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response3.json().get('entry')))

# references can also reference resources on other servers

Conditions for patient 17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f:
{'coding': [{'system': 'http://snomed.info/sct', 'code': '10509002', 'display': 'Acute bronchitis (disorder)'}], 'text': 'Acute bronchitis (disorder)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '195662009', 'display': 'Acute viral pharyngitis (disorder)'}], 'text': 'Acute viral pharyngitis (disorder)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '68235000', 'display': 'Nasal congestion (finding)'}], 'text': 'Nasal congestion (finding)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '267102003', 'display': 'Sore throat symptom (finding)'}], 'text': 'Sore throat symptom (finding)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '248595008', 'display': 'Sputum finding (finding)'}], 'text': 'Sputum finding (finding)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '84229001', 'display': 'Fatigue (finding)'}], 'text': 'Fatigue (finding)'}
{'coding': [{'system':

## Section 4: Chaining and includes

In [24]:
# Chaining

# where reference parameters get really interesting is when you want to query one resource type based on a property of another resource to which its linked
# for example, here is a search for Type II Diabetes in female patients
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&' + 'patient:Patient.gender=female' + '&_count=1')
peek('Type II Diabetes in female patients:   \n' + str(response.text), 100)

Type II Diabetes in female patients:   
{
    "resourceType": "Bundle",
    "id": "a8bf3e28-c44c-4272-aec2-72fa86fc0c05",
    "type": "searchset",
    "total": 600,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition?_count=1&code=http://snomed.info/sct%7C44054006&patient:Patient.gender=female&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition?_count=1&code=http://snomed.info/sct%7C44054006&patient:Patient.gender=female&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Condition/17598ca24de-8a64595d-1f7f-4a9d-af9b-9623bbd3db6d",
            "resource": {
                "resourceType": "

In [25]:
# Reverse chaining

# references can be searched the other way around via the "_has" parameter
response = get(base + '/Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_count=1')
peek('Patients with Type II Diabetes:   \n' + response.text)

Patients with Type II Diabetes:   
{
    "resourceType": "Bundle",
    "id": "e32adaca-4d16-49aa-a808-712fae72a115",
    "type": "searchset",
    "total": 1063,
    "link": [
        {
            "relation": "self",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_has:Condition:patient:code=http://snomed.info/sct%7C44054006&_page=1"
        },
        {
            "relation": "next",
            "url": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient?_count=1&_has:Condition:patient:code=http://snomed.info/sct%7C44054006&_page=2"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/Patient/17598c0ea1f-f838725e-91e0-4a98-9eb6-f3e7323e9dfa",
            "resource": {
                "resourceType": "Patient",
              

In [26]:
# Includes

# its also possible to get a resource and its related resources back in a single query
response = get(base + '/Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient' + '&_count=2')
peek('Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))


Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:

Total: 1063
Condition: 17598c0ea20-7b251c64-b10a-4942-b72b-3014af79875f
Condition: 17598c6d269-b8bd24cc-0280-48f8-a835-db262812387e
Patient: 17598c0ea1f-f838725e-91e0-4a98-9eb6-f3e7323e9dfa
Patient: 17598c6d269-6c393947-fbeb-44b7-9ec7-fbaede13bc8a


In [27]:
# Reverse Includes

response = get(base + '/Patient?gender=female' + '&' + '_revinclude=Condition:patient' + '&_count=2')
peek('Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))

Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:

Total: 17136
Patient: 17598bf36c7-fedcedc3-b78c-4688-82ef-622e0cc71b22
Patient: 17598bf4a5d-185e291b-d0e3-42d3-9b70-eeae60debeec
Condition: 17598bf4a5e-b9e99052-25a6-4400-a8b8-da48b8cbb22d
Condition: 17598bf4a65-ee99ca87-65c3-4a14-900f-02e90ec4db4a
Condition: 17598bf4a66-49d7afde-a7f1-42e4-858d-990f49a2f1ca
Condition: 17598bf4a66-39d8080e-27ee-488d-9e35-59275a486457
Condition: 17598bf4a6f-b87ac7b6-6faf-49b5-95fd-3eddf4f7494c
Condition: 17598bf4a6f-ec9abd6f-ee59-4f71-8fa5-c737ac1be63b
Condition: 17598bf4a70-b98f87d7-d469-4b3d-987e-02b78336b82e
Condition: 17598bf4a70-6e7537b4-1a90-4a3c-842c-6c5c1db45ad3
Condition: 17598bf4a70-ca048f40-02a3-4c24-a1b6-f53ffb3c1667
Condition: 17598bf36c8-5800bd67-4c08-4a68-8968-054c670ef2a6
Condition: 17598bf36c8-7d2861ad-e607-4122-90b3-74265240aaca
Condition: 17598bf36c9-d53628c0-8fc4-4570-bb9a-25af21328712
Condition: 17598bf36c9-9dec87f6-1a92-4b8

## Section 5: Putting it together

In [28]:
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&_count=1')
print('Patients with Type II Diabetes:  ' + str(response.json().get('total')))

Patients with Type II Diabetes:  1063


In [29]:
# SNOMED concepts for comorbidities of Type II Diabetes
#coronary heart disease (CHD), 53741008
#chronic kidney disease (CKD), 709044004
#atrial fibrillation, 49436004
#stroke, 230690007
#hypertension, 38341003
#heart failure, 84114007
#peripheral vascular disease (PVD), 400047006
#rheumatoid arthritis, 69896004
#Malignant neoplasm, primary (morphologic abnormality), 86049000
#Malignant neoplastic disease (disorder), 363346000
#osteoporosis, 64859006
#depression, 35489007
#asthma, 195967001
#chronic obstructive pulmonary disease (COPD), 13645005
#dementia, 52448006
#severe mental illness (SMI), 391193001
#epilepsy, 84757009
#hypothyroidism, 40930008
#learning disability, 1855002

print('CHD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '53741008').json().get('total')))
print('CKD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '709044004').json().get('total')))
print('AFib: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '49436004').json().get('total')))
print('stroke: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '230690007').json().get('total')))
print('hypertension: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '38341003').json().get('total')))
print('heart failure: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84114007').json().get('total')))
print('PVD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '400047006').json().get('total')))
print('arthritis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '69896004').json().get('total')))
print('cancer: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '86049000').json().get('total') + get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '363346000').json().get('total')))
print('osteoporosis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '64859006').json().get('total')))
print('depression: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '35489007').json().get('total')))
print('asthma: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '195967001').json().get('total')))
print('COPD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '13645005').json().get('total')))
print('dementia: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '52448006').json().get('total')))
print('SMI: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '391193001').json().get('total')))
print('epilepsy: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84757009').json().get('total')))
print('hypothyroidism: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '40930008').json().get('total')))
print('learning disability: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '1855002').json().get('total')))

CHD: 			1059
CKD: 			0
AFib: 			359
stroke: 		1008
hypertension: 		0
heart failure: 		84
PVD: 			0
arthritis: 		48
cancer: 		0
osteoporosis: 		762
depression: 		0
asthma: 		24
COPD: 			0
dementia: 		0
SMI: 			0
epilepsy: 		546
hypothyroidism: 	0
learning disability: 	0


In [30]:
# Patients with Type II Diabetes *and* comorbidities
hasDiabetes = base + '/Patient?_elements=id&_has:Condition:patient:code=http://snomed.info/sct|44054006'

def printPatientsWithComorbidity(conceptId):
    responseJSON = get(hasDiabetes + '&_has:Condition:patient:code=http://snomed.info/sct|' + conceptId).json()
    print('Total: ' + str(responseJSON.get('total')))
    if 'entry' in responseJSON:
        for entry in responseJSON.get('entry'):
            print(entry.get('resource').get('id'), end=", ")
    print('\n')

print('CHD:')
printPatientsWithComorbidity('53741008')

print('AFib:')
printPatientsWithComorbidity('49436004')

print('stroke:')
printPatientsWithComorbidity('230690007')

print('heart failure:')
printPatientsWithComorbidity('84114007')

print('arthritis:')
printPatientsWithComorbidity('69896004')

print('osteoporosis:')
printPatientsWithComorbidity('64859006')

print('asthma:')
printPatientsWithComorbidity('195967001')

print('epilepsy:')
printPatientsWithComorbidity('84757009')

CHD:
Total: 30
1759a06f11a-77b7ee76-ae8d-47a8-9a8e-17e7278e4137, 1759bccd5dc-a4192660-b224-4d88-acf6-bfb538fc0052, 1759c042626-da786c63-fcd4-4a86-aaec-54ec28458861, 1759c7179ab-cdc927fb-ec72-4c10-961e-a424bb8241f3, 1759c906bce-6fcc151d-240e-441f-aa3f-3a8123222d0c, 1759c9d0e32-10946dda-2ac9-4f8e-a6ba-eade9aba6ac8, 175b90993dc-7568b0d3-91ce-4862-be00-7cb99389f323, 175ba73ffc5-86182577-c52f-4c24-b09c-acfdc0043b8c, 175babfca99-31bcaafc-876a-453a-905c-b0482607b2c5, 175bb745e4f-1f3f83a8-6f6c-43da-adf1-fdc5440d88b5, 

AFib:
Total: 23
1759b62a9bc-650fc722-8bb1-4a81-827c-95b73a83d7b4, 1759be696c1-fd6cfa13-8e69-454f-ad73-d3b27a051f6f, 1759c1d61ec-500b2890-fcc9-428c-86c2-bcb15a5cbaa5, 1759c4dfa60-d50fbfcb-dda0-4669-82c8-42e73c5bb239, 1759c76bdcd-dfc226a9-37b6-4a72-b303-1613ed7ec838, 175b9f4c2a0-e971ee19-f55e-4319-85bd-57737b23ba26, 175ba948672-bc9ba458-81df-4def-8064-da02406e9f62, 175ba9518ee-29c6291a-ae29-4c62-8283-6c5a1264c404, 175ba9ca813-0a7f0565-f701-4c66-9d11-7b2754dddf6d, 175baa492a2-c7306

## Section 6: Bulk export

In [31]:
# To perform deeper analysis of the data, it can be useful to export some or all of the data into "bulk fhir" format
export_response = get(base + '/$export' + '?' + '_type=Patient,Condition')
print('Response code: ' + str(export_response.status_code))
print(export_response.headers)

Response code: 202
{'Date': 'Mon, 16 Nov 2020 20:55:51 GMT', 'Content-Length': '0', 'Connection': 'keep-alive', 'Content-Location': 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open/$bulkdata-status?job=C44eJARlx28_nw%2B_tH%2Bjlg', 'Content-Language': 'en-US', 'Strict-Transport-Security': 'max-age=86400; includeSubDomains'}


In [32]:
import time

# poll the status endpoint (returned in the Content-Location header of the $export response)
status_response = get(export_response.headers['Content-Location'])
print('Response code: ' + str(status_response.status_code))

while(status_response.status_code != 200):
    time.sleep(20)
    status_response = get(export_response.headers['Content-Location'])
    print('Response code: ' + str(status_response.status_code))
    
print('Response body: ' + str(status_response.text))

Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 202
Response code: 200
Response body: 
{
    "transactionTime": "2020-11-16T21:05:33.645Z",
    "request": "$export?_type=Patient,Condition",
    "requiresAccessToken": false,
    "output": [
        {
            "type": "Patient",
            "url": "https://s3.us-east.cloud-object-storage.appdomain.cloud/fhir-export-2fbaaea5-68b5-4775-8a40-c1eb3c4f12d5/yZc5xIahhrgSlxGgTZS33AWxMN9Dmr6sAHALAlIZLog=/Patient_1.ndjson",
            "count": 26000
    

In [33]:
# retrieve one of the NDJSON files and view the first 25 rows
ndjson = get(status_response.json().get('output')[1].get('url'))
peek(ndjson.text)

{"resourceType":"Patient","id":"175bd3f85e2-755eaef7-0bdb-4122-94fe-7090f2b829b7","meta":{"versionId":"1","lastUpdated":"2020-11-12T16:17:38.282Z","profile":["http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"]},"text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: v2.6.1-45-gc8c88a7e\n .   Person seed: -3906462213851746096  Population seed: 1604618056762</div>"},"extension":[{"extension":[{"url":"ombCategory","valueCoding":{"system":"urn:oid:2.16.840.1.113883.6.238","code":"2054-5","display":"Black or African American"}},{"url":"text","valueString":"Black or African American"}],"url":"http://hl7.org/fhir/us/core/StructureDefinition/us-core-race"},{"extension":[{"url":"ombCategory","valueCoding":{"system":"urn:oid:2.16.840.1.113883.6.238","code":"2186-5","display":"Non Hispanic or Latino"}},{"url":"text","valueString":"Non Hispanic or Latino"}],"ur

In [34]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

---
FHIR® is the registered trademark of HL7 and is used with the permission of HL7. Use of the FHIR trademark does not constitute endorsement of this product by HL7.